In [ ]:
import json
from datetime import datetime

# File paths to store data
PRODUCT_DB = "products.json"
SUPPLIER_DB = "suppliers.json"

# Product class
class Product:
    def __init__(self, product_id, name, price, stock_level, reorder_point, supplier_id=None):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.stock_level = stock_level
        self.reorder_point = reorder_point
        self.supplier_id = supplier_id

    def update_stock(self, quantity):
        self.stock_level += quantity

    def to_dict(self):
        return {
            "product_id": self.product_id,
            "name": self.name,
            "price": self.price,
            "stock_level": self.stock_level,
            "reorder_point": self.reorder_point,
            "supplier_id": self.supplier_id
        }

# Supplier class
class Supplier:
    def __init__(self, supplier_id, name, contact_info):
        self.supplier_id = supplier_id
        self.name = name
        self.contact_info = contact_info
        self.orders = []

    def add_order(self, product_id, quantity, date):
        self.orders.append({"product_id": product_id, "quantity": quantity, "date": date})

    def to_dict(self):
        return {
            "supplier_id": self.supplier_id,
            "name": self.name,
            "contact_info": self.contact_info,
            "orders": self.orders
        }

# Inventory class
class Inventory:
    def __init__(self):
        self.products = []
        self.suppliers = []

    def load_data(self):
        try:
            # Load products from the product database
            with open(PRODUCT_DB, 'r') as f:
                products_data = json.load(f)
                self.products = [Product(**data) for data in products_data]
            
            # Load suppliers from the supplier database
            with open(SUPPLIER_DB, 'r') as f:
                suppliers_data = json.load(f)
                self.suppliers = [Supplier(**data) for data in suppliers_data]
        except FileNotFoundError:
            print("Data files not found, starting with an empty inventory.")

    def save_data(self):
        # Save products to product database
        with open(PRODUCT_DB, 'w') as f:
            json.dump([product.to_dict() for product in self.products], f)

        # Save suppliers to supplier database
        with open(SUPPLIER_DB, 'w') as f:
            json.dump([supplier.to_dict() for supplier in self.suppliers], f)

    def add_product(self, name, price, stock_level, reorder_point, supplier_id):
        product_id = len(self.products) + 1
        new_product = Product(product_id, name, price, stock_level, reorder_point, supplier_id)
        self.products.append(new_product)
        self.save_data()

    def update_product(self, product_id, name=None, price=None, stock_level=None, reorder_point=None):
        product = next((p for p in self.products if p.product_id == product_id), None)
        if product:
            if name: product.name = name
            if price: product.price = price
            if stock_level is not None: product.stock_level = stock_level
            if reorder_point is not None: product.reorder_point = reorder_point
            self.save_data()

    def delete_product(self, product_id):
        self.products = [p for p in self.products if p.product_id != product_id]
        self.save_data()

    def view_inventory(self):
        return self.products

    def reorder_stock(self, product_id, quantity):
        product = next((p for p in self.products if p.product_id == product_id), None)
        if product:
            product.update_stock(quantity)
            self.save_data()

    def generate_report(self):
        report = []
        for product in self.products:
            if product.stock_level <= product.reorder_point:
                report.append({
                    "product_name": product.name,
                    "current_stock": product.stock_level,
                    "reorder_point": product.reorder_point
                })
        return report

    def add_supplier(self, name, contact_info):
        supplier_id = len(self.suppliers) + 1
        new_supplier = Supplier(supplier_id, name, contact_info)
        self.suppliers.append(new_supplier)
        self.save_data()

    def view_suppliers(self):
        return self.suppliers

    def add_order_from_supplier(self, supplier_id, product_id, quantity):
        supplier = next((s for s in self.suppliers if s.supplier_id == supplier_id), None)
        if supplier:
            supplier.add_order(product_id, quantity, str(datetime.now()))
            self.save_data()

# Example Usage
if __name__ == "__main__":
    inventory = Inventory()
    inventory.load_data()

    # Add suppliers
    inventory.add_supplier("Supplier A", "1234 Street, City, Country")
    inventory.add_supplier("Supplier B", "5678 Avenue, City, Country")

    # Add products
    inventory.add_product("Product 1", 10.99, 100, 20, 1)
    inventory.add_product("Product 2", 15.49, 50, 15, 2)

    # View inventory
    print("Current Inventory:")
    for product in inventory.view_inventory():
        print(f"ID: {product.product_id}, Name: {product.name}, Stock Level: {product.stock_level}")

    # Update product stock
    inventory.update_product(1, stock_level=80)

    # Reorder stock
    inventory.reorder_stock(1, 50)

    # View product details after update
    print("\nUpdated Inventory:")
    for product in inventory.view_inventory():
        print(f"ID: {product.product_id}, Name: {product.name}, Stock Level: {product.stock_level}")

    # Generate reorder report
    print("\nReorder Report:")
    report = inventory.generate_report()
    for item in report:
        print(item)

    # Add an order from a supplier
    inventory.add_order_from_supplier(1, 1, 30)

    # View supplier orders
    print("\nSupplier Orders:")
    for supplier in inventory.view_suppliers():
        print(f"Supplier: {supplier.name}")
        for order in supplier.orders:
            print(order)

    # Delete a product
    inventory.delete_product(2)
    
    # Save data
    inventory.save_data()
